In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2000&cat=FieldGoalsAttempted2&agg=PerGame"

In [3]:
for year in years:
    with open("attempted_two_pointers_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [4]:
dfs = []
for year in years:
    with open("attempted_two_pointers_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    attempted_data = pd.read_html(str(player_table))[0]
    attempted_data["Year"] = year
    dfs.append(attempted_data)

In [5]:
attempted_data = pd.concat(dfs)
attempted_data

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"FORD, ALPHONSOFORD, A.",PERPeristeriPeristeri,12,14.33,8.42,58.72%,2001
1,2,"TOMASEVIC, DEJANTOMASEVIC, D.",BUDBuducnostBuducnost,12,13.67,8.75,64.02%,2001
2,3,"RICH, JAMEILRICH, J.",LUGLugano SnakesLugano Snakes,10,12.80,6.50,50.78%,2001
3,4,"LIADELIS, PANAGIOTISLIADELIS, P.",PBCPAOK ThessalonikiPAOK Thessaloniki,13,12.46,6.62,53.09%,2001
4,5,"BATISTE, MIKEBATISTE, M.",CHASpirou CharleroiRegion Wallone Spirou Charl...,10,12.20,6.30,51.64%,2001
...,...,...,...,...,...,...,...,...
45,46,"ADAMS, THOMASADAMS, THOMAS",OVAOvarense AerosolesOvarense Aerosoles,10,5.90,2.70,45.76%,2021
46,46,"TAYLOR, ERICTAYLOR, ERIC",LONHaribo London TowersHaribo London Towers,10,5.90,2.60,44.07%,2021
47,46,"KRUNIC, DARKOKRUNIC, D.",ZADZadarZadar,10,5.90,3.30,55.93%,2021
48,46,"BAZAREVICH, SERGEYBAZAREVICH, SERGEY",PETSaint Petersburg LionsSaint Petersburg Lions,10,5.90,3.00,50.85%,2021


In [6]:
engine = create_engine('sqlite:///attempted_two_pointers_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
attempted_data.to_sql(
    name = "attempted_two_pointers_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [8]:
attempted_two_pointers_leaders_select = session.execute('select * from "attempted_two_pointers_leaders"')
attempted_two_pointers_leaders_select.all()

[(0, 1, 'FORD, ALPHONSOFORD, A.', 'PERPeristeriPeristeri', 12, 14.33, 8.42, '58.72%', 2001),
 (1, 2, 'TOMASEVIC, DEJANTOMASEVIC, D.', 'BUDBuducnostBuducnost', 12, 13.67, 8.75, '64.02%', 2001),
 (2, 3, 'RICH, JAMEILRICH, J.', 'LUGLugano SnakesLugano Snakes', 10, 12.8, 6.5, '50.78%', 2001),
 (3, 4, 'LIADELIS, PANAGIOTISLIADELIS, P.', 'PBCPAOK ThessalonikiPAOK Thessaloniki', 13, 12.46, 6.62, '53.09%', 2001),
 (4, 5, 'BATISTE, MIKEBATISTE, M.', 'CHASpirou CharleroiRegion Wallone Spirou Charleroi', 10, 12.2, 6.3, '51.64%', 2001),
 (5, 6, 'FUCKA, GREGORFUCKA, G.', 'FORPAF BolognaPAF Bologna', 17, 11.47, 6.76, '58.97%', 2001),
 (6, 7, 'HAMILTON, DERECKHAMILTON, D.', 'PETSaint Petersburg LionsSaint Petersburg Lions', 10, 11.4, 7.0, '61.40%', 2001),
 (7, 8, 'HILL, KRISHILL, K.', 'OVAOvarense AerosolesOvarense Aerosoles', 10, 11.1, 7.2, '64.86%', 2001),
 (8, 9, 'BECIROVIC, SANIBECIROVIC, S.', 'LJUUnion Olimpija LjubljanaUnion Olimpija Ljubljana', 15, 10.07, 4.4, '43.71%', 2001),
 (9, 10, 'MATTHE